# Registration of confocal images of Drosophila abdomens 

This notebook runs the entire pipeline to register the confocal images of fly abdomens.
The analysis is divided in few steps. Most of the code to perform each step is packed into python functions saved in files in the src folder and its subfolders. 

The results of each step of the analysis are saved in a series of ordered subfolders of the data folder.

In [2]:
# import packages 

import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import sys

root_dir = os.path.join(os.getcwd(), '..')
sys.path.append(root_dir)

## 1. Automated segmentation of the abdomens

In [5]:
# Importing the preprocessing function from the preprocess.py file:

from src.preprocess import preprocess_and_segment_images

raw_data_folder = "../../data_2/01_raw"
destination_folder = "../../data_2/02_preprocessed"

#downscaling of confocal stacks along z,x,y:
downscaling = (1, 2.5, 2.5)

# bit depth of raw images (depending on detector settings may be 8, 12 or 16)
bit_depth = 12

# name of the excel file containing the list of images to be used in the analysis. 
db_name = 'DatasetInformation.xlsx'

preprocess_and_segment_images(raw_data_folder, destination_folder, downscaling, bit_depth, only_on_new_files = True, database_filename = db_name)

Preprocessing of raw images in progress:


100%|███████████████████████████████████████████████████████████████████| 6/6 [06:52<00:00, 68.71s/it]


## 2. Manual registration of the segmented 3D images


In [6]:
# Importing the registration function from the registration.py file:

from src.registration import registration_of_abdomens_3D

"""
# USER INPUT IS REQUIRED, a series of 3D viewer windows will pop-up for each image in the dataset.

For each couple of images:
1)  Please pick at least three correspondences using [shift + left click]
    Press [shift + right click] to undo point picking)
2)  After picking points, press 'Q' to close the window
3)  Repeat points 1 and 2 for the second image.
4)  The next window will show the result of the registration, press 'Q' to close the window

"""

read_folder = "../../data_2/02_preprocessed"
destination_folder = "../../data_2/03_registered"

reference_fly_filename = "../../data_2/References_and_masks/C1_Reference_iso.tif"
abdomen_mask_file = "../../data_2/References_and_masks/Reference_abdomen_mask_iso.tif"

df_name = "DatasetInformation.xlsx"

preprocessed_df = pd.read_excel(os.path.join(read_folder,df_name))
registration_of_abdomens_3D(preprocessed_df, read_folder, reference_fly_filename, abdomen_mask_file, destination_folder, only_on_new_files = True)

Registration of 3D stacks in progress:


100%|███████████████████████████████████████████████████████████████████| 6/6 [08:36<00:00, 86.00s/it]


## 3. Automated 2D projection of registered 3D images 

In [ ]:
from src.projection import projections_of_abdomens

read_folder = "../../data_2/03_registered"
destination_folder = "../../data_2/04_projected"
landmark_folder = "../../data_2/05_landmarks/data"
abdomen_mask_file = "../../data_2/References_and_masks/Reference_abdomen_mask_iso_thick.tif"
abdomen_shape_reference_file = "../../data_2/References_and_masks/Reference_abdomen_mask_iso.tif"

df_name = "DatasetInformation.xlsx"
df = pd.read_excel(os.path.join(read_folder,df_name))
projections_of_abdomens(df, read_folder, destination_folder, landmark_folder, abdomen_mask_file, abdomen_shape_reference_file, crop_x=300, crop_y = 320)

Projection of registered 3D stacks to 2D images in progress:


 67%|██████████████████████████████               | 4/6 [00:38<00:20, 10.17s/it]

##  4. Annotation and warping of the 2D projected images

After the projection from 3D to 2D, images are already roughly aligned. However, the alignment up to this point was only performed by applying a global rescaling and rigid rotations/translations of the original stacks.

To fine tune the registration we need to manually place landmarks on the 2D projected images and apply an elastic deformation of the images.

Run the image annotation software, open the annotation project, add the new images found in the folder 05_landmarks/data to the project.

Annotate the new images and run the registration, save the results in the folder 06_warped.

## 5. Final touch: smoothing and masking


In [3]:
from src.masking_smoothing import preprocess_registered_images

destination_folder = "../../data_2/07_masked_and_smooth"
read_folder = "../../data_2/06_warped"
mask = '../../data_2/References_and_masks/2D_mask_abdomen_vertical-1.tif'
database_registered_images = "../../data_2/06_warped/dataframe_info.csv"
database_info = "../../data_2/04_projected/DatasetInformation.xlsx"

preprocess_registered_images(read_folder, destination_folder, database_registered_images, database_info, mask_filename = mask, smoothing_sigma=4, binning=1)

FileNotFoundError: [Errno 2] No such file or directory: '../../data_2/06_warped/dataframe_info.csv'